### https://dacon.io/competitions/official/236355/overview/description

In [119]:
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.read_csv('./train.csv')
X_train, X_test, y_train, y_test = train_test_split(train.drop(columns=['ID', 'SUBCLASS']), train['SUBCLASS'])

In [120]:
import xgboost as xgb
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
labelEncoder = LabelEncoder()

In [121]:
y_train_encoded = labelEncoder.fit_transform(y_train)
y_test_encoded = labelEncoder.transform(y_test)
for i, label in enumerate(labelEncoder.classes_):
    print(f'{label} => {i}')
    

ACC => 0
BLCA => 1
BRCA => 2
CESC => 3
COAD => 4
DLBC => 5
GBMLGG => 6
HNSC => 7
KIPAN => 8
KIRC => 9
LAML => 10
LGG => 11
LIHC => 12
LUAD => 13
LUSC => 14
OV => 15
PAAD => 16
PCPG => 17
PRAD => 18
SARC => 19
SKCM => 20
STES => 21
TGCT => 22
THCA => 23
THYM => 24
UCEC => 25


In [122]:
categorical_columns = X_train.select_dtypes(include=['object', 'category']).columns
ordinalEncoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_train_encoded = X_train.copy()
X_train_encoded[categorical_columns] = ordinalEncoder.fit_transform(X_train[categorical_columns])
X_train_encoded

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
3526,101.0,34.0,20.0,0.0,37.0,113.0,84.0,90.0,149.0,89.0,...,126.0,53.0,19.0,20.0,96.0,49.0,38.0,35.0,26.0,33.0
1244,101.0,34.0,20.0,0.0,37.0,113.0,84.0,90.0,149.0,89.0,...,126.0,53.0,19.0,20.0,96.0,49.0,38.0,35.0,26.0,33.0
5627,101.0,34.0,20.0,0.0,37.0,113.0,84.0,90.0,149.0,89.0,...,126.0,43.0,19.0,20.0,96.0,49.0,38.0,35.0,26.0,33.0
5410,101.0,34.0,20.0,0.0,37.0,113.0,84.0,90.0,149.0,89.0,...,126.0,53.0,19.0,20.0,96.0,49.0,38.0,35.0,10.0,33.0
4990,101.0,34.0,20.0,0.0,37.0,113.0,84.0,90.0,149.0,89.0,...,126.0,53.0,19.0,20.0,96.0,49.0,38.0,35.0,26.0,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6091,101.0,34.0,20.0,0.0,37.0,113.0,84.0,90.0,149.0,89.0,...,126.0,53.0,19.0,20.0,96.0,49.0,38.0,35.0,26.0,33.0
4187,101.0,34.0,20.0,0.0,37.0,113.0,84.0,90.0,149.0,89.0,...,126.0,53.0,19.0,20.0,96.0,49.0,38.0,35.0,26.0,33.0
2825,101.0,34.0,20.0,0.0,37.0,113.0,84.0,90.0,149.0,89.0,...,126.0,53.0,19.0,20.0,96.0,49.0,38.0,35.0,26.0,33.0
3035,101.0,34.0,20.0,0.0,37.0,113.0,84.0,90.0,149.0,89.0,...,126.0,53.0,19.0,20.0,96.0,49.0,38.0,35.0,26.0,33.0


In [143]:
model = xgb.XGBClassifier(
    n_estimators=200,
    learning_rate=0.01,
    max_depth=6,
    random_state=42,
    verbosity=1
)
model.fit(X_train_encoded, y_train_encoded)

/opt/homebrew/anaconda3/envs/mlclass/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [15:25:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "eval_matrics", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [124]:
X_test_encoded = X_test.copy()
X_test_encoded[categorical_columns] = ordinalEncoder.transform(X_test_encoded[categorical_columns])

In [125]:
preds = model.predict(X_test_encoded)
original_label = labelEncoder.inverse_transform(preds)

In [126]:
pred_train = model.predict(X_train_encoded)
label = labelEncoder.inverse_transform(pred_train)

In [135]:
type(y_train.to_numpy())

numpy.ndarray

In [133]:
type(label)

numpy.ndarray

In [136]:
from sklearn.metrics import f1_score
train_f1 = f1_score(y_train.to_numpy(), label, average='macro')
test_f1 = f1_score(y_test.to_numpy(), original_label, average='macro')

print(f'학습 데이터 f1: {train_f1}')
print(f'테스트 데이터 f1: {test_f1}')


학습 데이터 f1: 0.7439913698810128
테스트 데이터 f1: 0.3010205565316561


In [137]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.50)
reduce_X = pca.fit_transform(X_train_encoded)

model.fit(reduce_X, y_train_encoded)



/opt/homebrew/anaconda3/envs/mlclass/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [15:18:31] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "eval_matrics", "n_estimator", "use_label_encoder", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [140]:
preds = model.predict(reduce_X)

preds = labelEncoder.inverse_transform(preds)
f1score = f1_score(y_train.to_numpy(), preds, average='macro')
print(f'train f1score: {f1score}')

test_X_encoded_pca = pca.transform(X_test_encoded)
preds = model.predict(test_X_encoded_pca)
original_label = labelEncoder.inverse_transform(preds)
f1score = f1_score(y_test.to_numpy(), original_label, average='macro')
print(f'test f1score: {f1score}')

train f1score: 0.8994935760949937
test f1score: 0.15746882924090944


In [44]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(
    random_state=42,
    n_estimators=200,
    bootstrap=True,
    max_depth=10
)
rf_model.fit(X_encoded, y_subclass)

RandomForestClassifier(max_depth=10, n_estimators=200, random_state=42)

In [45]:
preds = rf_model.predict(test_X_encoded)
original_label = labelEncoder.inverse_transform(preds)

submit = pd.read_csv('./sample_submission.csv')
submit['SUBCLASS'] = original_label

submit.to_csv('./baseline_rf.csv', encoding='UTF-8-sig', index=False)

rf 분류기

In [46]:
from sklearn.metrics import r2_score, accuracy_score
preds = rf_model.predict(X_encoded)
accuracy = accuracy_score(y_subclass, preds)
r2 = r2_score(y_subclass, preds)
print(f'accracy: {accuracy}\nr2: {r2}')

accracy: 0.3368811482019029
r2: -0.8834075327236162


XGBoost 분류기 100

In [47]:
preds = model.predict(X_encoded)
accuracy = accuracy_score(y_subclass, preds)
r2 = r2_score(y_subclass, preds)
print(f'accracy: {accuracy}\nr2: {r2}')

accracy: 0.6732785034671828
r2: 0.19492638524417583


XGBoost 분류기 300

In [50]:
preds = model.predict(X_encoded)
accuracy = accuracy_score(y_subclass, preds)
r2 = r2_score(y_subclass, preds)
print(f'accracy: {accuracy}\nr2: {r2}')

accracy: 0.6732785034671828
r2: 0.19492638524417583


In [63]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.80)
reduce_X = pca.fit_transform(X_encoded)

model.fit(reduce_X, y_subclass)
preds = model.predict(reduce_X)

accuracy = accuracy_score(y_subclass, preds)
r2 = r2_score(y_subclass, preds)
print(f'accracy: {accuracy}\nr2: {r2}')

/opt/homebrew/anaconda3/envs/mlclass/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [13:55:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "eval_matrics", "n_estimator", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


accracy: 0.9161425576519916
r2: 0.9558928255966902


In [65]:
test_X_encoded_pca = pca.transform(test_X_encoded)
preds = model.predict(test_X_encoded_pca)
original_label = labelEncoder.inverse_transform(preds)

submit = pd.read_csv('./sample_submission.csv')
submit['SUBCLASS'] = original_label

submit.to_csv('./baseline_pca80.csv', encoding='UTF-8-sig', index=False)

In [64]:
pca.n_components_

127

In [66]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
reduce_X = pca.fit_transform(X_encoded)

model.fit(reduce_X, y_subclass)
preds = model.predict(reduce_X)

accuracy = accuracy_score(y_subclass, preds)
r2 = r2_score(y_subclass, preds)
print(f'accracy: {accuracy}\nr2: {r2}')

pca.n_components_

/opt/homebrew/anaconda3/envs/mlclass/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [13:59:13] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "eval_matrics", "n_estimator", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


accracy: 0.9174326721496533
r2: 0.9581758543678327


501

In [67]:
test_X_encoded_pca = pca.transform(test_X_encoded)
preds = model.predict(test_X_encoded_pca)
original_label = labelEncoder.inverse_transform(preds)

submit = pd.read_csv('./sample_submission.csv')
submit['SUBCLASS'] = original_label

submit.to_csv('./baseline_pca95.csv', encoding='UTF-8-sig', index=False)

In [68]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.50)
reduce_X = pca.fit_transform(X_encoded)

model.fit(reduce_X, y_subclass)
preds = model.predict(reduce_X)

accuracy = accuracy_score(y_subclass, preds)
r2 = r2_score(y_subclass, preds)
print(f'accracy: {accuracy}\nr2: {r2}')

test_X_encoded_pca = pca.transform(test_X_encoded)
preds = model.predict(test_X_encoded_pca)
original_label = labelEncoder.inverse_transform(preds)

submit = pd.read_csv('./sample_submission.csv')
submit['SUBCLASS'] = original_label

submit.to_csv('./baseline_pca50.csv', encoding='UTF-8-sig', index=False)

/opt/homebrew/anaconda3/envs/mlclass/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [14:01:34] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "eval_matrics", "n_estimator", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


accracy: 0.814707305273343
r2: 0.7100582881122874
